# Prepare

In [1]:
import open3d as o3d
from open3d import visualization, io
import os
import copy
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import time
from pathlib import Path
from sklearn.cluster import DBSCAN
import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 01 Load point clouds

## Parameters Definition

In [2]:
Path_raw_PCDs = "./raw_pcds/"

## Funtion Definition

In [3]:
def filter_points_by_range(point_cloud, center= np.array([0.0, 0.0, 0.0]), threshold_x = [-7.0, 3.0], threshold_y = [-10.0, 10.0]):
    # extract points
    points = np.asarray(point_cloud.points)

    # filter
    mask = ((threshold_x[0] <= points[:, 0]) & (points[:, 0] <= threshold_x[1])) & ((threshold_y[0] <= points[:, 1]) & (points[:, 1] <= threshold_y[1]))
    filtered_points = points[mask]

    # create new point_cloud
    filtered_point_cloud = o3d.geometry.PointCloud()
    filtered_point_cloud.points = o3d.utility.Vector3dVector(filtered_points)

    return filtered_point_cloud

In [4]:
def load_point_clouds(path, filtering = True):
    point_clouds = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith(".pcd"):
            full_path = os.path.join(path, filename)
            point_cloud = o3d.io.read_point_cloud(full_path)
            if filtering:
              filtered_point_cloud = filter_points_by_range(point_cloud)
              point_clouds.append(filtered_point_cloud)
            else:
              point_clouds.append(point_cloud)
    return point_clouds

## Apply Loading point clouds

### Unfiltered

In [25]:
# Load point clouds
point_clouds_unfiltered = load_point_clouds(Path_raw_PCDs, filtering= False)
print(f"{len(point_clouds_unfiltered)} unfiltered oint clouds have been loaded.")

60 unfiltered oint clouds have been loaded.


In [26]:
all_point_clouds_unfiltered = point_clouds_unfiltered[0]
for i in range(1, len(point_clouds_unfiltered)):
  all_point_clouds_unfiltered = all_point_clouds_unfiltered + point_clouds_unfiltered[i]
o3d.io.write_point_cloud("01_Loading_Filtering_Point-clouds/all_point_clouds_unfiltered.pcd", all_point_clouds_unfiltered)

True

### Filtered

In [7]:
# Load point clouds
point_clouds = load_point_clouds(Path_raw_PCDs)
print(f"{len(point_clouds)} point clouds have been loaded.")

60 point clouds have been loaded.


In [28]:
all_point_clouds = point_clouds[0]
for i in range(1, len(point_clouds)):
  all_point_clouds = all_point_clouds + point_clouds[i]
o3d.io.write_point_cloud("01_Loading_Filtering_Point-clouds/all_point_clouds.pcd", all_point_clouds)

True

## Visualization

### Unfiltered

In [ ]:
all_point_clouds_unfiltered = io.read_point_cloud("01_Loading_Filtering_Point-clouds/all_point_clouds_unfiltered.pcd") # Read point cloud

In [ ]:
visualization.draw_geometries([all_point_clouds_unfiltered])    # Visualize point cloud 

### Filtered

In [5]:
all_point_clouds = io.read_point_cloud("01_Loading_Filtering_Point-clouds/all_point_clouds.pcd") # Read point cloud

In [6]:
visualization.draw_geometries([all_point_clouds])    # Visualize point cloud 

# 02 ICP

## Function Definition

In [7]:
# ICP registration for sequential point clouds
def register_point_clouds(point_clouds, threshold = 0.05):
    registered_clouds = [point_clouds[0]]
    for i in range(1, len(point_clouds)):
        source = point_clouds[i]
        target = registered_clouds[-1]

        icp_result = o3d.pipelines.registration.registration_icp(
            source, target, max_correspondence_distance= threshold,
            estimation_method= o3d.pipelines.registration.TransformationEstimationPointToPoint()
        )
        transformed_source = source.transform(icp_result.transformation)
        registered_clouds.append(transformed_source)
    return registered_clouds

## Parameters Definition

In [8]:
icp_threshold = 0.05

## Apply ICP Algorithm

In [31]:
# Register point clouds
registered_point_clouds = register_point_clouds(point_clouds, icp_threshold)
print("Point clouds were registered.")

Point clouds were registered.


## Save 

In [32]:
all_registered_point_clouds = registered_point_clouds[0]
for i in range(1, len(registered_point_clouds)):
  all_registered_point_clouds = all_registered_point_clouds + registered_point_clouds[i]
o3d.io.write_point_cloud("02_Point-clouds_Registration_ICP/all_registered_point_clouds.pcd", all_registered_point_clouds)

True

## Visualization 

In [9]:
all_registered_point_clouds = io.read_point_cloud("02_Point-clouds_Registration_ICP/all_registered_point_clouds.pcd") # Read point cloud

In [10]:
visualization.draw_geometries([all_registered_point_clouds])    # Visualize point cloud 

# 03 Motion Detection

## Function Definition

In [11]:
# Find neighbor for a point
# Return: motion vector with shape (3, 1)
def find_neighbor(point: np.ndarray, target_point_cloud: o3d.geometry.Geometry.Type.PointCloud):
    target_points = np.asarray(target_point_cloud.points)

    tree = o3d.geometry.KDTreeFlann(target_point_cloud)

    _, idx, _ = tree.search_knn_vector_3d(point, 1)
    if len(idx) > 0:
        # a neighbor has been found
        closest_point = target_points[idx[0]]
        motion_vector = (closest_point - point).reshape((1, 3))
    else:
        # No movement if no neighbor found
        motion_vector = np.zeros((1, 3))
    return motion_vector

In [12]:
# Calculation motion vectors
# Return: dataFrames
def calculate_motion_vectors(registered_point_clouds, sliding_window_size = 5):
    print(f"{len(registered_point_clouds) - sliding_window_size} frames to be processed. The first {sliding_window_size} frames couldn't be processed.")

    dfs = []
    for count_source_point_cloud in range(sliding_window_size, len(registered_point_clouds)):
        start_time = time.time()
        source_point_cloud = np.asarray(registered_point_clouds[count_source_point_cloud].points)

        # create dictionary
        dic = {
            "source_point": [],
        }
        for count_sliding_window in range(sliding_window_size):
            dic[f"motion_vector_{count_sliding_window + 1}"] = []

        for point in source_point_cloud:
            dic["source_point"].append(point.reshape((1, 3)))
            for count_target_point_cloud in range(sliding_window_size):
                # for each point in source point cloud: find neighbor
                target_point_cloud = registered_point_clouds[count_source_point_cloud - count_target_point_cloud - 1]
                dic[f"motion_vector_{str(int(sliding_window_size - count_target_point_cloud))}"].append(find_neighbor(point, target_point_cloud))

        # convert to df
        df = pd.DataFrame(dic)
        dfs.append(df)

        print(f"Frame {count_source_point_cloud} finished, {time.time() - start_time} seconds used, {len(registered_point_clouds) - count_source_point_cloud - 1} frames left ......")

    return dfs

In [13]:
def calculate_magnitude(vector):
    return np.linalg.norm(vector)

In [14]:
def assign_label(magnitude, threshold = 0.05):
    if magnitude >= threshold:
        return True  # motion labeled
    else:
        return False  # no motion labeled

In [15]:
def load_dfs(path):
    dfs = []
    for filename in sorted(os.listdir(path)):
        if filename.endswith(".pkl"):
            full_path = os.path.join(path, filename)
            df = pd.read_pickle(full_path)
            dfs.append(df.sort_values(by=["avg_motion_vector_mag"], ascending= False))
    return dfs

## Parameters definition

In [16]:
motion_threshold = 0.05

## Sliding window size = 1

In [17]:
sliding_window_size = 1

### Calculate motion vectors, average motion vectors, average motion vectors magnitude, labels

#### motion vectors

In [40]:
dfs = calculate_motion_vectors(registered_point_clouds, sliding_window_size)

59 frames to be processed. The first 1 frames couldn't be processed.
Frame 1 finished, 13.921431303024292 seconds used, 58 frames left ......
Frame 2 finished, 13.701355457305908 seconds used, 57 frames left ......
Frame 3 finished, 13.540055990219116 seconds used, 56 frames left ......
Frame 4 finished, 13.321339130401611 seconds used, 55 frames left ......
Frame 5 finished, 15.295138597488403 seconds used, 54 frames left ......
Frame 6 finished, 13.548601150512695 seconds used, 53 frames left ......
Frame 7 finished, 14.196861743927002 seconds used, 52 frames left ......
Frame 8 finished, 13.507101058959961 seconds used, 51 frames left ......
Frame 9 finished, 14.219792604446411 seconds used, 50 frames left ......
Frame 10 finished, 13.503329038619995 seconds used, 49 frames left ......
Frame 11 finished, 13.343739748001099 seconds used, 48 frames left ......
Frame 12 finished, 14.573378086090088 seconds used, 47 frames left ......
Frame 13 finished, 13.395508289337158 seconds used, 

#### Average motion vectors

In [47]:
for df in dfs:
    temp_avg_motion_vectors = df["motion_vector_1"]
    for count_sliding_window in range(1, sliding_window_size):
        temp_avg_motion_vectors = temp_avg_motion_vectors + df[f"motion_vector_{count_sliding_window + 1}"]
    df["avg_motion_vector"] = temp_avg_motion_vectors / float(sliding_window_size)

#### Average motion vector magnitudes

In [48]:
for df in dfs:
    df["avg_motion_vector_mag"] = df["avg_motion_vector"].apply(calculate_magnitude)

In [49]:
# sorting by avg_motion_vector_mag
for count_df in range(len(dfs)):
    dfs[count_df] = dfs[count_df].sort_values(by=["avg_motion_vector_mag"], ascending= False)

#### Assign labels

In [50]:
for df in dfs:
    df["motion"] = df["avg_motion_vector_mag"].apply(assign_label, threshold= motion_threshold)

### Save dfs in file

In [57]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_{:0>1}/".format(sliding_window_size)
Path(Path_dfs).mkdir(parents=True, exist_ok=True)

In [58]:
for count_df in range(len(dfs)):
    df = dfs[count_df]
    df.to_pickle(os.path.join(Path_dfs, "df_{:0>3}.pkl".format(count_df)))

### Load motion vectors, average motion vectors, average motion vectors magnitude, labels from dfs in File

In [ ]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_{:0>1}/".format(sliding_window_size)
dfs = load_dfs(Path_dfs)

## Sliding window size = 5

In [18]:
sliding_window_size = 5

### Calculate motion vectors, average motion vectors, average motion vectors magnitude, labels

#### motion vectors

In [60]:
dfs = calculate_motion_vectors(registered_point_clouds, sliding_window_size)

55 frames to be processed. The first 5 frames couldn't be processed.
Frame 5 finished, 68.18129014968872 seconds used, 54 frames left ......
Frame 6 finished, 67.50930404663086 seconds used, 53 frames left ......
Frame 7 finished, 68.07767033576965 seconds used, 52 frames left ......
Frame 8 finished, 67.4980034828186 seconds used, 51 frames left ......
Frame 9 finished, 68.74377465248108 seconds used, 50 frames left ......
Frame 10 finished, 67.22436356544495 seconds used, 49 frames left ......
Frame 11 finished, 67.31480956077576 seconds used, 48 frames left ......
Frame 12 finished, 68.60805606842041 seconds used, 47 frames left ......
Frame 13 finished, 67.01027584075928 seconds used, 46 frames left ......
Frame 14 finished, 69.02768278121948 seconds used, 45 frames left ......
Frame 15 finished, 68.82853198051453 seconds used, 44 frames left ......
Frame 16 finished, 66.24284887313843 seconds used, 43 frames left ......
Frame 17 finished, 68.54877710342407 seconds used, 42 frames 

#### Average motion vectors

In [61]:
for df in dfs:
    temp_avg_motion_vectors = df["motion_vector_1"]
    for count_sliding_window in range(1, sliding_window_size):
        temp_avg_motion_vectors = temp_avg_motion_vectors + df[f"motion_vector_{count_sliding_window + 1}"]
    df["avg_motion_vector"] = temp_avg_motion_vectors / float(sliding_window_size)

#### Average motion vector magnitudes

In [62]:
for df in dfs:
    df["avg_motion_vector_mag"] = df["avg_motion_vector"].apply(calculate_magnitude)

In [63]:
# sorting by avg_motion_vector_mag
for count_df in range(len(dfs)):
    dfs[count_df] = dfs[count_df].sort_values(by=["avg_motion_vector_mag"], ascending= False)

#### Assign labels

In [64]:
for df in dfs:
    df["motion"] = df["avg_motion_vector_mag"].apply(assign_label, threshold= motion_threshold)

### Save dfs in file

In [65]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_{:0>1}/".format(sliding_window_size)
Path(Path_dfs).mkdir(parents=True, exist_ok=True)

In [66]:
for count_df in range(len(dfs)):
    df = dfs[count_df]
    df.to_pickle(os.path.join(Path_dfs, "df_{:0>3}.pkl".format(count_df)))

### Load motion vectors, average motion vectors, average motion vectors magnitude, labels from dfs in File

In [19]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_{:0>1}/".format(sliding_window_size)
dfs = load_dfs(Path_dfs)

## Visualization 

#### Sliding window size = 1

In [ ]:
def assign_color(label):
    if label == True:
        return [1.0, 0.0, 0.0]  # red
    else:
        return [0.5, 0.5, 0.5]  # gray

In [ ]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_1"
num_example_frame = 30
df_example_frame = pd.read_pickle(os.path.join(Path_dfs, "df_{:0>3}.pkl".format(num_example_frame)))

In [ ]:
df_example_frame["color"] = df_example_frame["motion"].apply(assign_color)

In [ ]:
points = np.vstack(df_example_frame["source_point"].values) 
colors = np.vstack(df_example_frame["color"].values) 
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)
point_cloud.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([point_cloud])

#### Sliding window size = 5

In [20]:
def assign_color(label):
    if label == True:
        return [1.0, 0.0, 0.0]  # red
    else:
        return [0.5, 0.5, 0.5]  # gray

In [21]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_5"
num_example_frame = 30
df_example_frame = pd.read_pickle(os.path.join(Path_dfs, "df_{:0>3}.pkl".format(num_example_frame)))

In [22]:
df_example_frame["color"] = df_example_frame["motion"].apply(assign_color)

In [23]:
points = np.vstack(df_example_frame["source_point"].values) 
colors = np.vstack(df_example_frame["color"].values) 
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)
point_cloud.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([point_cloud])

# 04 DBSCAN

## Function Definition 

In [24]:
color_map = {
    0: [1, 0, 0],     # Cluster 0 - Red
    1: [0, 1, 0],     # Cluster 1 - Green
    2: [0, 0, 1],     # Cluster 2 - Blue
    3: [1, 1, 0],     # Cluster 3 - Yellow
    4: [1, 0, 1],     # Cluster 4 - Magenta
    5: [0, 1, 1],     # Cluster 5 - Cyan
}

In [25]:
def dbscan_clustering(df: pd.DataFrame, threshold_motion= 0.05, eps= 1.0, min_samples= 10):
    # Points with motion
    mask_motion = df.loc[:, "avg_motion_vector_mag"] >= threshold_motion
    
    points_motion = np.vstack(df[mask_motion].loc[:, "source_point"].values)
    avg_motion_vector = np.vstack(df[mask_motion].loc[:, "avg_motion_vector"].values)
    features = points_motion
    # features = np.hstack((points_motion, avg_motion_vector))
    
    # DBSCAN only for points, that were with motion-detected marked
    clustering = DBSCAN(eps= eps, min_samples= min_samples).fit(features)
    labels = clustering.labels_
    
    num_clusters = len(set(labels)) - (1 if -1 in labels else 0)    # noise points ignored
    num_noise = np.sum(labels == -1)
    print(f"{num_clusters} clusters found")
    # print(f"Points of noise: {num_noise}")
    
    # Find the cluster with the highest degree of aggregation (the cluster with the most points)
    unique_labels = set(labels)
    label_counts = {}
    
    for label in unique_labels:
        label_counts[label] = np.sum(labels == label)
    
    # for label, count in label_counts.items():
    #     print(f"Cluster {label} contains: {count} points")

    max_cluster_label = max(label_counts, key= label_counts.get)
    
    
    # Points without motion
    mask_no_motion = df.loc[:, "avg_motion_vector_mag"] < threshold_motion
    points_no_motion = np.vstack(df[mask_no_motion].loc[:, "source_point"].values)
    
    
    # Assign colors for different clusters
    unique_labels = set(labels)
    colors_points_motion = []
    
    for label in labels:
        if label == -1:
            colors_points_motion.append([0.5, 0.5, 0.5])  # gray for noise points
        else:
            colors_points_motion.append(color_map.get(label, [0.5, 0.5, 0.5]))
    
    ## for points_no_motion
    colors_points_no_motion = []
    for i in range(points_no_motion.shape[0]):
        colors_points_no_motion.append([0.5, 0.5, 0.5])  # gray
    
    
    # Visualization
    cloud_points_motion = o3d.geometry.PointCloud()
    cloud_points_motion.points = o3d.utility.Vector3dVector(points_motion)
    cloud_points_motion.colors = o3d.utility.Vector3dVector(colors_points_motion)
    
    cloud_points_no_motion = o3d.geometry.PointCloud()
    cloud_points_no_motion.points = o3d.utility.Vector3dVector(points_no_motion)
    cloud_points_no_motion.colors = o3d.utility.Vector3dVector(colors_points_no_motion)
    
    return cloud_points_motion + cloud_points_no_motion

## Parameters 

In [26]:
Path_dfs = "./03_Motion_Detection_Optical-flow/sliding_windowsize_5"
Path_pcds_clustered = "./04_Clustering_Object-Extraction_DBSCAN"

## Apply DBSCAN 

In [ ]:
for count_df in range(0, 54):
    df = pd.read_pickle(os.path.join(Path_dfs, "df_{:0>3}.pkl".format(count_df)))
    point_cloud = dbscan_clustering(df, eps= 0.4, min_samples= 25, threshold_motion= 0.05)
    o3d.io.write_point_cloud(os.path.join(Path_pcds_clustered, "point_cloud_{:0>3}.pcd".format(count_df)), point_cloud)

1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found
1 clusters found


## Visualization 

In [27]:
num_example_point_cloud = 30
example_point_cloud = io.read_point_cloud(os.path.join(Path_pcds_clustered, "point_cloud_{:0>3}.pcd".format(num_example_point_cloud))) # Read point cloud
o3d.visualization.draw_geometries([example_point_cloud])

# 05 Visualization 

In [28]:
Path_pcds_clustered = "./04_Clustering_Object-Extraction_DBSCAN"
view_config = "saved_view.json"

In [29]:
vis= o3d.visualization.Visualizer()
vis.create_window(width= 1440, height= 900)

point_cloud = io.read_point_cloud(os.path.join(Path_pcds_clustered, "point_cloud_{:0>3}.pcd".format(0))) # Read first point cloud
vis.add_geometry(point_cloud)
vis.update_renderer()

try:
    with open(view_config, 'r') as f:
        view_params = json.load(f)
    
    # PinholeCameraParameters 
    ctr = vis.get_view_control()
    camera_params = ctr.convert_to_pinhole_camera_parameters()
    
    # extrinsic 
    extrinsic = np.array(view_params['extrinsic']).reshape((4, 4))
    camera_params.extrinsic = extrinsic
    
    # intrinsic 
    intrinsic = o3d.camera.PinholeCameraIntrinsic()
    intrinsic.intrinsic_matrix = np.array(view_params["intrinsic"]['intrinsic_matrix']).reshape((3, 3))
    intrinsic.width = view_params["intrinsic"]['width']
    intrinsic.height = view_params["intrinsic"]['height']
    camera_params.intrinsic = intrinsic
    
    # Apply paramters
    ctr.convert_from_pinhole_camera_parameters(camera_params, allow_arbitrary= True)
    ctr.set_zoom(0.5)
    ctr.set_lookat([-1, 0, 0])
    
    print("View parameters loaded and applied.")
except Exception as e:
    print(f"Failed to load view parameters: {e}")


for num_point_cloud in range(1, 54):
    temo_point_cloud = io.read_point_cloud(os.path.join(Path_pcds_clustered, "point_cloud_{:0>3}.pcd".format(num_point_cloud))) # Read point cloud
    point_cloud.points = temo_point_cloud.points
    point_cloud.colors = temo_point_cloud.colors
    vis.update_geometry(point_cloud)
    
    vis.poll_events()
    vis.update_renderer()
    time.sleep(0.1)  # Delay

vis.destroy_window()

View parameters loaded and applied.
